In [49]:
# Import dependencies
import os, sys
from PIL import Image
from IPython.display import display
from IPython.display import Image as _Imgdis
import numpy as np
# from time import time
# from time import sleep
# from scipy import ndimage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# import keras 


In [68]:
# Fetch dataset
# ArborioFolder = './Rice_Image_Dataset/Arborio/'
# BasmatiFolder = './Rice_Image_Dataset/Basmati/'
# IpsalaFolder = './Rice_Image_Dataset/Ipsala/'
# JasmineFolder = './Rice_Image_Dataset/Jasmine/'
# KaracadagFolder = './Rice_Image_Dataset/Karacadag/'

# arborio_files = [f for f in os.listdir(ArborioFolder) if os.path.isfile(os.path.join(ArborioFolder, f))]
labels = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']
X = []
y = []

for label in labels:
    folder = './Rice_Image_Dataset/' + label + '/'
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    for i in range(100):
        img = Image.open(folder + "/" + files[i])
        img_array = np.asarray(img)
        X.append(img_array)
        y.append(label)
        # img_back = Image.fromarray(img_array)
        # img_back.show()

X = np.asarray(X)
Xr = X.reshape(500, 62500, 3)
# print(np.shape(Xr))
# img_1 = Image.fromarray(Xr[0])
# img_2 = Image.fromarray(Xr[1])

# print(len(X))
# print(y)
# display(_Imgdis(filename=ArborioFolder + "/" + arborio_files[i], width=240, height=320))

In [72]:
# Split dataset
X_train, X_val, y_train, y_val = train_test_split(Xr, y, test_size=0.8, random_state=42)
print(np.ndim(X_train))

3


In [70]:
# fit the PCA
# N = 
pca = PCA() # keep all features?
X_train_pca = pca.fit_transform(X_train)

# plot the explained variances
fig, ax1 = plt.subplots(figsize=(12, 5))
color = 'tab:blue'
ax1.bar(1+np.arange(N), pca.explained_variance_ratio_, color=color)
ax1.set_xticks(1+np.arange(N, step=2))
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylabel("Explained variance ratio", color=color)
ax1.set_xlabel("Generated feature")

ax2 = ax1.twinx()
color = 'tab:red'
ax2.tick_params(axis='y', labelcolor=color)
ax2.plot(1+np.arange(N), np.cumsum(pca.explained_variance_ratio_), color=color)
ax2.set_ylabel("Cumulative explained variance ratio", color=color)
fig.tight_layout()
plt.show()


ValueError: Found array with dim 4. PCA expected <= 2.

In [ ]:
https://www.kaggle.com/code/lgmoneda/from-image-files-to-numpy-arrays
https://www.geeksforgeeks.org/how-to-convert-images-to-numpy-array/
